In [64]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import math
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_percentage_error
import holidays
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

def mean_absolute_percentage_error(y_true, y_pred):
    return (abs(y_true - y_pred) / y_true).mean() * 100


df = pd.read_csv("monthly_data.csv")

In [65]:
import pandas as pd

# Assuming your dataframe is named 'df'
mapping = {
    120004096: "Product1",
    120009814: "Product2",
    120009816: "Product3",
    120010342: "Product4",
    120010566: "Product5",
    120010970: "Product6",
    120011556: "Product7",
    120011782: "Product8",
    120012154: "Product9",
    120012606: "Product10",
    120014486: "Product11",
    120014488: "Product12",
    120015842: "Product13",
    120015996: "Product14"
}

df['ID_material'] = df['ID_material'].map(mapping)

In [66]:
# df = df[df["ID_material"] == "Product1"]
# df = df.drop(columns="ID_material")
# df.head(3)

In [67]:
categorical_cols = ['MonthName',"Season"]
numerical_cols = ["Number_Of_Holidays_In_That_Month_SPAIN","GDP_Past_Quarter_Change","PreviousUnemploymentQuarterAmmount","Months_Since_Material_Release","Absolute_Previous GDP Quarter Change"]
# Filter the data for the last month of 2022 (August)
test_df = df[(df['Year'] == 2022) & (df['MonthName'].isin(['August', 'September']))]

# Separate the features and target variable for the test set
X_test = test_df.drop(['Units'], axis=1)
y_test = test_df['Units']

# Filter the data for training (excluding the last month of 2022)
train_df = df[~((df['Year'] == 2022) & (df['MonthName'].isin(['August', 'September'])))]

# Separate the features and target variable for training
X_train = train_df.drop(['Units'], axis=1)
y_train = train_df['Units']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_cols),
        ('num', StandardScaler(), numerical_cols)
    ])

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [68]:
y_test = y_test+1

In [69]:
import optuna
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
def objective(trial):
     # Define the hyperparameters to search over
     params = {
         'n_estimators': trial.suggest_int('n_estimators', 50, 1000, step=50),
         'max_depth': trial.suggest_int('max_depth', 2, 20),
         'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
         'gamma': trial.suggest_float('gamma', 0, 20),
         'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
         "eval_metric": mean_squared_error     
     }

    
     # Train an XGBoost classifier with the specified hyperparameters
     clf = xgb.XGBRegressor(**params)
     clf.fit(X_train_processed, y_train)
    
     # Calculate the auc on the test set
     y_pred = clf.predict(X_test_processed)
     rmse = mean_squared_error(y_test, y_pred,squared=False)
    
     # Return the accuracy as the objective value for Optuna to optimize
     return rmse


# Use Optuna to search for the best hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)
# Print the best hyperparameters found by Optuna
print(f'Best parameters: {study.best_params}')
print(f'Best RMSE: {study.best_value:.3f}')

[I 2023-06-21 16:17:41,597] A new study created in memory with name: no-name-1a622067-751b-455c-ab56-890b3a1a31b6
[I 2023-06-21 16:17:41,966] Trial 0 finished with value: 4720.053292881821 and parameters: {'n_estimators': 200, 'max_depth': 14, 'learning_rate': 0.031252832875078566, 'subsample': 0.8367176533470411, 'colsample_bytree': 0.8157448136536144, 'gamma': 4.150467819437593, 'min_child_weight': 12}. Best is trial 0 with value: 4720.053292881821.
[I 2023-06-21 16:17:42,834] Trial 1 finished with value: 4450.1539025392985 and parameters: {'n_estimators': 750, 'max_depth': 20, 'learning_rate': 0.008445431832644528, 'subsample': 0.8558273971797592, 'colsample_bytree': 0.5003747543846614, 'gamma': 14.259849201565483, 'min_child_weight': 16}. Best is trial 1 with value: 4450.1539025392985.
[I 2023-06-21 16:17:42,981] Trial 2 finished with value: 4286.6379242640505 and parameters: {'n_estimators': 150, 'max_depth': 6, 'learning_rate': 0.009175920782863028, 'subsample': 0.841701942828866

Best parameters: {'n_estimators': 1000, 'max_depth': 17, 'learning_rate': 0.00164606854275803, 'subsample': 0.6758923778178398, 'colsample_bytree': 0.6561287436730823, 'gamma': 3.0616560671791433, 'min_child_weight': 18}
Best RMSE: 4088.648


In [70]:
#best_para = {'n_estimators': 250, 'max_depth': 4, 'learning_rate': 0.007106555945980472, 'subsample': 0.7737423542367182, 'colsample_bytree': 0.5131340580856684, 'gamma': 13.17031716280021, 'min_child_weight': 7}

test_setProduct1

In [71]:
import pandas  as pd
import numpy   as np
import xgboost as xgb

regressor=xgb.XGBRegressor(**study.best_params)
regressor.fit(X_train_processed, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6561287436730823, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=3.0616560671791433, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.00164606854275803, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=17, max_leaves=None,
             min_child_weight=18, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [72]:
import pandas as pd

# Get unique values of ID_material column
unique_materials = df['ID_material'].unique()

# Filter rows for each unique material and specific year/month range
for material in unique_materials:
    test_set = df[(df['ID_material'] == material) & (df['Year'] == 2022) & (df['MonthName'].isin(['August', 'September']))]
    globals()[f"test_set{material}"] = test_set.copy()  # Create a separate DataFrame for each test set




predictions_df = pd.DataFrame(columns=['Material', 'MAPE'])

# Iterate over each unique material
for material in unique_materials:
    # Get the corresponding test set for the material
    test_set = globals()[f"test_set{material}"]
    X_test = test_set.drop(columns="Units",axis=0)
    
    test_processed = preprocessor.transform(X_test)
    
    # Get the predictions for the test set
    predictions = regressor.predict(test_processed)
    
    # Create a DataFrame for the predictions
    predictions_material = pd.DataFrame({'Test': test_set['Units'], 'Prediction': predictions})
    
    # Calculate MAPE
    mape = np.mean(np.abs((predictions_material['Test'] - predictions_material['Prediction']) / predictions_material['Test'])) * 100
    
    # Store material and MAPE in predictions_df
    predictions_df = predictions_df.append({'Material': material, 'MAPE': mape}, ignore_index=True)

    # Optional: Store predictions in a separate DataFrame
    globals()[f"predictions_{material}"] = predictions_material.copy()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_10340\649432351.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({'Material': material, 'MAPE': mape}, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_10340\649432351.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({'Material': material, 'MAPE': mape}, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_10340\649432351.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions_df = predictions_df.append({'Material': material, 'MAPE': mape}, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_10340\649432351.py:34: FutureWarning: The frame.append

In [73]:
mape_df = pd.DataFrame(columns=['Material', 'MAPE_Sum'])


In [78]:
predictions_Product7

,Test,Prediction
642,1530.0,6058.311523
653,414.0,5925.620605


In [79]:
test_setProduct7

,ID_material,Year,Season,MonthName,Units,Number_Of_Holidays_In_That_Month_SPAIN,GDP_Past_Quarter_Change,PreviousUnemploymentQuarterAmmount,Months_Since_Material_Release,Absolute_Previous GDP Quarter Change
642,Product7,2022,Summer,August,1530.0,1,2.5,2919.4,34,2.5
653,Product7,2022,Autumn,September,414.0,0,2.5,2919.4,35,2.5


In [76]:
for material in unique_materials:
    # Get the corresponding test set and prediction DataFrame for the material
    test_set = globals()[f"test_set{material}"]
    predictions_material = globals()[f"predictions_{material}"]
    
    # Calculate the sum of values for test set and predictions
    test_sum = test_set['Units'].sum()
    pred_sum = predictions_material['Prediction'].sum()
    
    # Calculate MAPE of sum of values
    mape_sum = np.abs((test_sum - pred_sum) / test_sum) * 100
    
    # Store material and MAPE of sum in mape_df
    mape_df = mape_df.append({'Material': material, 'MAPE_Sum': mape_sum}, ignore_index=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_10340\1779927863.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_df = mape_df.append({'Material': material, 'MAPE_Sum': mape_sum}, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_10340\1779927863.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_df = mape_df.append({'Material': material, 'MAPE_Sum': mape_sum}, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_10340\1779927863.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_df = mape_df.append({'Material': material, 'MAPE_Sum': mape_sum}, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_10340\1779927863.py:14: FutureWarning: The frame.append method is dep

In [80]:
mape_df["MAPE_Sum"].mean()

155.1136430809024